In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from catboost import CatBoostClassifier
from numpy import*
from category_encoders.target_encoder import TargetEncoder

In [27]:
df = pd.read_csv('train/train_meta.csv')
#общие метаданные
df_local = pd.read_csv('train/train_gts_final.csv')
#метки локализации инфаркта/его отсутствия
df_myocard = pd.read_csv('train/train_gts.csv')
#инфаркт/его отсутствие

### Преобразование в строковые, сложение и применение целевого кодирования

In [265]:
pred_value = df_local['перегородочный'].astype(str) + '_' + df_local['передний'].astype(str) \
    + '_' + df_local['боковой'].astype(str) + '_' + df_local['передне-боковой'].astype(str) \
    + '_' + df_local['передне-перегородочный'].astype(str) + '_' + df_local['нижний'].astype(str) \
    + '_' + df_local['норма'].astype(str)
df_local.insert(0, 'pred_value', pred_value)
#объединение предсказываемых величин

target_encoder_pred_value = TargetEncoder()
target_encoder_pred_value.fit(df_local.pred_value, df_gts['myocard'])
target_encoded_pred_value = target_encoder_pred_value.transform(df_local.pred_value)
df_local.pred_value = target_encoded_pred_value.astype('float')

df_local.drop(['перегородочный', 'передний', 'боковой', 'передне-боковой', 'передне-перегородочный',
               'нижний', 'норма', 'record_name'], axis= 1 , inplace= True )

In [266]:
df_local.value_counts()

pred_value
0.000000      1686
0.999999       159
0.999338        91
0.949465        47
0.904343        40
0.481877        28
0.697033        25
0.429481        11
0.383278         8
0.321480         3
0.311357         2
0.301934         1
dtype: int64

### Преобразование в строковые и сложение

In [284]:
df_local = pd.read_csv('train/train_gts_final.csv')
#метки локализации инфаркта/его отсутствия

In [28]:
pred_value = df_local['перегородочный'].astype(str) + df_local['передний'].astype(str) \
    + df_local['боковой'].astype(str) + df_local['передне-боковой'].astype(str) \
    + df_local['передне-перегородочный'].astype(str) + df_local['нижний'].astype(str) \
    + df_local['норма'].astype(str)
df_local.insert(0, 'pred_value', pred_value)
#объединение предсказываемых величин

df_local.drop(['перегородочный', 'передний', 'боковой', 'передне-боковой', 'передне-перегородочный',
               'нижний', 'норма', 'record_name'], axis= 1 , inplace= True )

In [29]:
df_local.value_counts()

pred_value
0000001       1686
0000010        159
0000100         91
1000000         47
0100000         40
0000110         25
0001000         14
0100010         14
1000010         11
0001010          8
0001100          3
0010000          2
0101010          1
dtype: int64

In [30]:
combo_info = df['weight'].astype(str) + '_' + df['height'].astype(str) \
    + '_' + df['age'].astype(str) + '_' + df['sex'].astype(str)
df.insert(0, 'combo_info', combo_info)
#объединение данных веса, роста, возраста и пола(во избежание переобучения)

target_encoder_combo_info = TargetEncoder()
target_encoder_combo_info.fit(df.combo_info, df_myocard)
target_encoded_combo_info = target_encoder_combo_info.transform(df.combo_info)
df.combo_info = target_encoded_combo_info.astype('float')

ValueError: Unexpected input shape: (2101, 2)

In [31]:
df.drop(['record_name', 'weight', 'height', 'age', 'sex'], axis= 1 , inplace= True )

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df, df_local, train_size=0.9, random_state=0)

In [26]:
cat_meta = CatBoostClassifier(depth=8, learning_rate=0.05, verbose=False)
#loss_function='MultiClass',
cat_meta.fit(X_train, y_train)

TypeError: unhashable type: 'numpy.ndarray'

In [18]:
f1_score(y_train, cat_meta.predict(X_train), average = 'macro')

0.06875126629296954

In [19]:
f1_score(y_test, cat_meta.predict(X_test), average = 'macro')

0.08594594594594594